Advanced Usage
Filtering by Subject or Object

python
Copy code
plot_svo_graph(svos, subject_filter='John')
Exporting Hypergraphs

python
Copy code
hypergraphs = export_hypergraphs(svos)
print(hypergraphs)


In [ ]:
plot_svo_graph
Visualizes the SVO relationships in a graph.

Parameters:

df (DataFrame): The DataFrame containing SVO data.
subject_filter (str): (Optional) Filter graph by a specific subject.
object_filter (str): (Optional) Filter graph by a specific object.
Example:

python
Copy code
plot_svo_graph(svos, subject_filter='Alice')
Other Functions
export_hypergraphs(df)
export_subj(df)
export_obj(df)
export_verb(df)